In [32]:
#import matplotlib with pdf as backend
import matplotlib 
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import wfdb 
import os
import numpy as np
import math
import sys
import scipy.stats as st
import glob, os
from os.path import basename


import tensorflow as tf
from keras.layers import Dense,Activation,Dropout
from keras.layers import LSTM,Bidirectional #could try TimeDistributed(Dense(...))
from keras.models import Sequential, load_model
from keras import optimizers,regularizers
from keras.layers.normalization import BatchNormalization
import keras.backend.tensorflow_backend as KTF
np.random.seed(0)

In [10]:
import aiohttp
import asyncio

data_path=r'F:\data_analysis_tool\DL4Learning\MY_ARRANGEMENT\signal\signal_proj\ECG_data'

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.read()

async def download(url):
    print(url)
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url)
        with open(os.path.join(data_path,os.path.basename(url)), "wb") as code:
            code.write(html)    

In [40]:
# functions
def get_ecg_data(datfile): 
	## convert .dat/q1c to numpy arrays
	recordname=os.path.basename(datfile).split(".dat")[0]
	recordpath=os.path.dirname(datfile)
	cwd=os.getcwd()
	os.chdir(recordpath) ## somehow it only works if you chdir. 

	annotator='q1c'
	annotation = wfdb.rdann(recordname, extension=annotator, sampfrom=0,sampto = None, pbdir=None)
	Lstannot=list(zip(annotation.sample,annotation.symbol,annotation.aux_note))

	FirstLstannot=min( i[0] for i in Lstannot)
	LastLstannot=max( i[0] for i in Lstannot)-1
	print("first-last annotation:", FirstLstannot,LastLstannot)
	
	record = wfdb.rdsamp(recordname, sampfrom=FirstLstannot,sampto = LastLstannot) #wfdb.showanncodes()
	annotation = wfdb.rdann(recordname, annotator, sampfrom=FirstLstannot,sampto = LastLstannot) ## get annotation between first and last. 
	annotation2 = wfdb.Annotation(recordname='sel32', extension='niek', sample=(annotation.sample-FirstLstannot), symbol = annotation.symbol, aux_note=annotation.aux_note)

	Vctrecord=np.transpose(record.p_signals)
	VctAnnotationHot=np.zeros( (6,len(Vctrecord[1])), dtype=np.int)
	VctAnnotationHot[5]=1 ## inverse of the others 

	VctAnnotations=list(zip(annotation2.sample,annotation2.symbol)) ## zip coordinates + annotations (N),(t) etc)
	#print(VctAnnotations)
	for i in range(len(VctAnnotations)):
		#print(VctAnnotations[i]) # Print to display annotations of an ecg
		try: 
			
			if VctAnnotations[i][1]=="p":
				if VctAnnotations[i-1][1]=="(":
					pstart=VctAnnotations[i-1][0]
				if VctAnnotations[i+1][1]==")":
					pend=VctAnnotations[i+1][0]
				if VctAnnotations[i+3][1]=="N":
					rpos=VctAnnotations[i+3][0]
					if VctAnnotations[i+2][1]=="(":
						qpos=VctAnnotations[i+2][0]
					if VctAnnotations[i+4][1]==")":
						spos=VctAnnotations[i+4][0]	
					for ii in range(0,8): ## search for t (sometimes the "(" for the t  is missing  )
						if VctAnnotations[i+ii][1]=="t":
							tpos=VctAnnotations[i+ii][0]
							if VctAnnotations[i+ii+1][1]==")":
								tendpos=VctAnnotations[i+ii+1][0]
			# 				#print(ppos,qpos,rpos,spos,tendpos)
								VctAnnotationHot[0][pstart:pend]=1 #P segment
								VctAnnotationHot[1][pend:qpos]=1 #part "nothing" between P and Q, previously left unnanotated, but categorical probably can't deal with that
								VctAnnotationHot[2][qpos:rpos]=1 #QR
								VctAnnotationHot[3][rpos:spos]=1 #RS
								VctAnnotationHot[4][spos:tendpos]=1 #ST (from end of S to end of T)
								VctAnnotationHot[5][pstart:tendpos]=0 #tendpos:pstart becomes 1, because it is inverted above					
		except IndexError:
			pass
	
	Vctrecord=np.transpose(Vctrecord) # transpose to (timesteps,feat)
	VctAnnotationHot=np.transpose(VctAnnotationHot)
	os.chdir(cwd)
	return Vctrecord, VctAnnotationHot



def splitseq(x,n,o):
	#split seq; should be optimized so that remove_seq_gaps is not needed. 
	upper=math.ceil( x.shape[0] / n) *n
	print("splitting on",n,"with overlap of ",o,	"total datapoints:",x.shape[0],"; upper:",upper)
	for i in range(0,upper,n):
		#print(i)
		if i==0:
			padded=np.zeros( ( o+n+o,x.shape[1])   ) ## pad with 0's on init
			padded[o:,:x.shape[1]] = x[i:i+n+o,:]
			xpart=padded
		else:
			xpart=x[i-o:i+n+o,:]
		if xpart.shape[0]<i:

			padded=np.zeros( (o+n+o,xpart.shape[1])  ) ## pad with 0's on end of seq
			padded[:xpart.shape[0],:xpart.shape[1]] = xpart
			xpart=padded

		xpart=np.expand_dims(xpart,0)## add one dimension; so that you get shape (samples,timesteps,features)
		try:
			xx=np.vstack(  (xx,xpart) )
		except UnboundLocalError: ## on init
			xx=xpart
	print("output: ",xx.shape)
	return(xx)

def remove_seq_gaps(x,y):
	#remove parts that are not annotated <- not ideal, but quickest for now.
	window=150
	c=0
	cutout=[]
	include=[]
	print("filterering.")
	print("before shape x,y",x.shape,y.shape)
	for i in range(y.shape[0]):
		
		c=c+1
		if c<window :
			include.append(i)
		if sum(y[i,0:5])>0:
			c=0 
		if c >= window:
			#print ('filtering')
			pass
	x,y=x[include,:],y[include,:]
	print(" after shape x,y",x.shape,y.shape)
	return(x,y)


def normalizesignal(x):
	x=st.zscore(x, ddof=0)
	return x
def normalizesignal_array(x):
	for i in range(x.shape[0]):
		x[i]=st.zscore(x[i], axis=0, ddof=0)
	return x

def plotecg(x,y,begin,end):
	#helper to plot ecg
	plt.figure(1,figsize=(11.69,8.27))
	plt.subplot(211)
	plt.plot(x[begin:end,0])
	plt.subplot(211)
	plt.plot(y[begin:end,0])
	plt.subplot(211)
	plt.plot(y[begin:end,1])
	plt.subplot(211)
	plt.plot(y[begin:end,2])
	plt.subplot(211)
	plt.plot(y[begin:end,3])
	plt.subplot(211)
	plt.plot(y[begin:end,4])
	plt.subplot(211)
	plt.plot(y[begin:end,5])

	plt.subplot(212)
	plt.plot(x[begin:end,1])
	plt.show()

def plotecg_validation(x,y_true,y_pred,begin,end):
	#helper to plot ecg
	plt.figure(1,figsize=(11.69,8.27))
	plt.subplot(211)
	plt.plot(x[begin:end,0])
	plt.subplot(211)
	plt.plot(y_pred[begin:end,0])
	plt.subplot(211)
	plt.plot(y_pred[begin:end,1])
	plt.subplot(211)
	plt.plot(y_pred[begin:end,2])
	plt.subplot(211)
	plt.plot(y_pred[begin:end,3])
	plt.subplot(211)
	plt.plot(y_pred[begin:end,4])
	plt.subplot(211)
	plt.plot(y_pred[begin:end,5])

	plt.subplot(212)
	plt.plot(x[begin:end,1])
	plt.subplot(212)
	plt.plot(y_true[begin:end,0])
	plt.subplot(212)
	plt.plot(y_true[begin:end,1])
	plt.subplot(212)
	plt.plot(y_true[begin:end,2])
	plt.subplot(212)
	plt.plot(y_true[begin:end,3])
	plt.subplot(212)
	plt.plot(y_true[begin:end,4])
	plt.subplot(212)
	plt.plot(y_true[begin:end,5])
	
def LoaddDatFiles(datfiles):  
	for datfile in datfiles:
	    print(datfile)
	    if basename(datfile).split(".",1)[0] in exclude:
	    	continue
	    qf=os.path.splitext(datfile)[0]+'.q1c'
	    if os.path.isfile(qf):
	    	#print("yes",qf,datfile)
	    	x,y=get_ecg_data(datfile)
	    	x,y=remove_seq_gaps(x,y)

	    	x,y=splitseq(x,1000,150),splitseq(y,1000,150) ## create equal sized numpy arrays of n size and overlap of o 

	    	x = normalizesignal_array(x)
	    	## todo; add noise, shuffle leads etc. ?
	    	try: ## concat
	    		xx=np.vstack(  (xx,x) )
	    		yy=np.vstack(  (yy,y) )
	    	except NameError: ## if xx does not exist yet (on init)
	    		xx = x
	    		yy = y
	return(xx,yy)

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def get_session(gpu_fraction=0.8):
	#allocate % of gpu memory.
	num_threads = os.environ.get('OMP_NUM_THREADS')
	gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
	if num_threads:
		return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
	else:
		return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

def getmodel():
	model = Sequential()
	model.add(Dense(32,W_regularizer=regularizers.l2(l=0.01), input_shape=(seqlength, features)))
	model.add(Bidirectional(LSTM(32, return_sequences=True)))#, input_shape=(seqlength, features)) ) ### bidirectional ---><---
	model.add(Dropout(0.2))
	model.add(BatchNormalization())
	model.add(Dense(64, activation='relu',W_regularizer=regularizers.l2(l=0.01)))
	model.add(Dropout(0.2))
	model.add(BatchNormalization())
	model.add(Dense(dimout, activation='softmax'))
	adam = optimizers.adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
	model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy']) 
	print(model.summary())
	return(model)

In [51]:
num_threads,gpu_options

(None, per_process_gpu_memory_fraction: 0.8)

In [ ]:
##################################################################
##################################################################
import path
qtdbpath=sys.argv[1] ## first argument = qtdb database from physionet. 
qtdbpath=r'F:\data_analysis_tool\DL4Learning\MY_ARRANGEMENT\signal\signal_proj\ECG_data'+'/'
print(qtdbpath)
perct=0.81 #percentage training
percv=0.19 #percentage validation

exclude = set()
exclude.update(["sel35","sel36","sel37","sel50","sel102","sel104","sel221","sel232", "sel310"])# no P annotated:
##################################################################

# load data
datfiles=glob.glob(qtdbpath+"*.dat")
xxt,yyt=LoaddDatFiles(datfiles[ :round(len(datfiles)*perct) ]) # training data. 
xxt,yyt=unison_shuffled_copies(xxt,yyt) ### shuffle
xxv,yyv=LoaddDatFiles(datfiles[ -round(len(datfiles)*percv): ] ) ## validation data.
seqlength=xxt.shape[1]
features=xxt.shape[2]
dimout=yyt.shape[2]
print("xxv/validation shape: {}, Seqlength: {}, Features: {}".format(xxv.shape[0],seqlength,features))

In [100]:
xxv.shape,yyv[1]

((141, 1300, 2), array([[0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.]]))

<module 'tensorflow' from 'F:\\environment\\Anaconda3\\lib\\site-packages\\tensorflow\\__init__.py'>

In [53]:
# call keras/tensorflow and build lstm model 
KTF.set_session(get_session())
with tf.device('/cpu:0'): #switch to /cpu:0 to use cpu 
	if not os.path.isfile('model.h5'):
		model = getmodel() # build model
		model.fit(xxt, yyt, batch_size=32, epochs=100, verbose=1) # train the model
		model.save('model.h5')

	model = load_model('model.h5')
	score, acc = model.evaluate(xxv, yyv, batch_size=4, verbose=1)
	print('Test score: {} , Test accuracy: {}'.format(score, acc))
	
	# predict
	yy_predicted = model.predict(xxv) 

	# maximize probabilities of prediction.
	for i in range(yyv.shape[0]): 
		b = np.zeros_like(yy_predicted[i,:,:])
		b[np.arange(len(yy_predicted[i,:,:])), yy_predicted[i,:,:].argmax(1)] = 1
		yy_predicted[i,:,:] = b

	# plot: 
	with PdfPages('ecg.pdf') as pdf:
		for i in range( xxv.shape[0] ): 
			print (i)
			plotecg_validation(xxv[i,:,:],yy_predicted[i,:,:],yyv[i,:,:],0,yy_predicted.shape[1])  # top = predicted, bottom=true
			pdf.savefig()
			plt.close()

	#plotecg(xv[1,:,:],yv[1,:,:],0,yv.shape[1]) ## plot first seq

F:\environment\Anaconda3\lib\site-packages\ipykernel_launcher.py:215: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, input_shape=(1300, 2), kernel_regularizer=<keras.reg...)`
F:\environment\Anaconda3\lib\site-packages\ipykernel_launcher.py:219: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_regularizer=<keras.reg...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1300, 32)          96        
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1300, 64)          16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1300, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1300, 64)          256       
_________________________________________________________________
dense_2 (Dense)              (None, 1300, 64)          4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1300, 64)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 1300, 64)          256       
__________

F:\environment\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140


In [63]:
# as the first layer in a model
model = Sequential()
model.add(keras.layers.TimeDistributed(Dense(8), input_shape=(10, 16)))
# now model.output_shape == (None, 10, 8)
print(model.output_shape)
# as the first layer in a model

(None, 10, 8)


In [ ]:
md2 = Sequential()
# subsequent layers: no need for input_shape
md2.add(keras.layers.TimeDistributed(Dense(32)))
# now model.output_shape == (None, 10, 32)
print(md2.output_shape)

In [68]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length)])
X = seq.reshape(len(seq), 1, 1)
y = seq.reshape(len(seq), 1)
# define LSTM configuration
n_neurons = length
n_batch = length
n_epoch = 1000
# create LSTM
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())
# train LSTM
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=2)
# evaluate
result = model.predict(X, batch_size=n_batch, verbose=0)
for value in result:
    print('%.1f' % value)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 5)                 140       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 6         
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
 - 1s - loss: 0.2473
Epoch 2/1000
 - 0s - loss: 0.2455
Epoch 3/1000
 - 0s - loss: 0.2437
Epoch 4/1000
 - 0s - loss: 0.2420
Epoch 5/1000
 - 0s - loss: 0.2402
Epoch 6/1000
 - 0s - loss: 0.2384
Epoch 7/1000
 - 0s - loss: 0.2367
Epoch 8/1000
 - 0s - loss: 0.2349
Epoch 9/1000
 - 0s - loss: 0.2332
Epoch 10/1000
 - 0s - loss: 0.2315
Epoch 11/1000
 - 0s - loss: 0.2298
Epoch 12/1000
 - 0s - loss: 0.2281
Epoch 13/1000
 - 0s - loss: 0.2264
Epoch 14/1000
 - 0s - loss: 0.2247
Epoch 15/1000
 - 0s - loss: 0.2230
Epoch 16/1000
 - 0s

Epoch 215/1000
 - 0s - loss: 0.0524
Epoch 216/1000
 - 0s - loss: 0.0522
Epoch 217/1000
 - 0s - loss: 0.0520
Epoch 218/1000
 - 0s - loss: 0.0517
Epoch 219/1000
 - 0s - loss: 0.0515
Epoch 220/1000
 - 0s - loss: 0.0513
Epoch 221/1000
 - 0s - loss: 0.0511
Epoch 222/1000
 - 0s - loss: 0.0509
Epoch 223/1000
 - 0s - loss: 0.0507
Epoch 224/1000
 - 0s - loss: 0.0505
Epoch 225/1000
 - 0s - loss: 0.0503
Epoch 226/1000
 - 0s - loss: 0.0501
Epoch 227/1000
 - 0s - loss: 0.0499
Epoch 228/1000
 - 0s - loss: 0.0497
Epoch 229/1000
 - 0s - loss: 0.0495
Epoch 230/1000
 - 0s - loss: 0.0493
Epoch 231/1000
 - 0s - loss: 0.0492
Epoch 232/1000
 - 0s - loss: 0.0490
Epoch 233/1000
 - 0s - loss: 0.0488
Epoch 234/1000
 - 0s - loss: 0.0486
Epoch 235/1000
 - 0s - loss: 0.0485
Epoch 236/1000
 - 0s - loss: 0.0483
Epoch 237/1000
 - 0s - loss: 0.0481
Epoch 238/1000
 - 0s - loss: 0.0479
Epoch 239/1000
 - 0s - loss: 0.0478
Epoch 240/1000
 - 0s - loss: 0.0476
Epoch 241/1000
 - 0s - loss: 0.0475
Epoch 242/1000
 - 0s - loss:

Epoch 443/1000
 - 0s - loss: 0.0245
Epoch 444/1000
 - 0s - loss: 0.0244
Epoch 445/1000
 - 0s - loss: 0.0243
Epoch 446/1000
 - 0s - loss: 0.0242
Epoch 447/1000
 - 0s - loss: 0.0241
Epoch 448/1000
 - 0s - loss: 0.0240
Epoch 449/1000
 - 0s - loss: 0.0239
Epoch 450/1000
 - 0s - loss: 0.0238
Epoch 451/1000
 - 0s - loss: 0.0237
Epoch 452/1000
 - 0s - loss: 0.0236
Epoch 453/1000
 - 0s - loss: 0.0235
Epoch 454/1000
 - 0s - loss: 0.0234
Epoch 455/1000
 - 0s - loss: 0.0233
Epoch 456/1000
 - 0s - loss: 0.0231
Epoch 457/1000
 - 0s - loss: 0.0230
Epoch 458/1000
 - 0s - loss: 0.0229
Epoch 459/1000
 - 0s - loss: 0.0228
Epoch 460/1000
 - 0s - loss: 0.0227
Epoch 461/1000
 - 0s - loss: 0.0226
Epoch 462/1000
 - 0s - loss: 0.0225
Epoch 463/1000
 - 0s - loss: 0.0224
Epoch 464/1000
 - 0s - loss: 0.0223
Epoch 465/1000
 - 0s - loss: 0.0222
Epoch 466/1000
 - 0s - loss: 0.0221
Epoch 467/1000
 - 0s - loss: 0.0220
Epoch 468/1000
 - 0s - loss: 0.0219
Epoch 469/1000
 - 0s - loss: 0.0218
Epoch 470/1000
 - 0s - loss:

Epoch 671/1000
 - 0s - loss: 0.0049
Epoch 672/1000
 - 0s - loss: 0.0049
Epoch 673/1000
 - 0s - loss: 0.0048
Epoch 674/1000
 - 0s - loss: 0.0048
Epoch 675/1000
 - 0s - loss: 0.0047
Epoch 676/1000
 - 0s - loss: 0.0047
Epoch 677/1000
 - 0s - loss: 0.0046
Epoch 678/1000
 - 0s - loss: 0.0046
Epoch 679/1000
 - 0s - loss: 0.0046
Epoch 680/1000
 - 0s - loss: 0.0045
Epoch 681/1000
 - 0s - loss: 0.0045
Epoch 682/1000
 - 0s - loss: 0.0044
Epoch 683/1000
 - 0s - loss: 0.0044
Epoch 684/1000
 - 0s - loss: 0.0043
Epoch 685/1000
 - 0s - loss: 0.0043
Epoch 686/1000
 - 0s - loss: 0.0042
Epoch 687/1000
 - 0s - loss: 0.0042
Epoch 688/1000
 - 0s - loss: 0.0041
Epoch 689/1000
 - 0s - loss: 0.0041
Epoch 690/1000
 - 0s - loss: 0.0040
Epoch 691/1000
 - 0s - loss: 0.0040
Epoch 692/1000
 - 0s - loss: 0.0040
Epoch 693/1000
 - 0s - loss: 0.0039
Epoch 694/1000
 - 0s - loss: 0.0039
Epoch 695/1000
 - 0s - loss: 0.0038
Epoch 696/1000
 - 0s - loss: 0.0038
Epoch 697/1000
 - 0s - loss: 0.0037
Epoch 698/1000
 - 0s - loss:

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(5, 10),return_sequences=False))
#model.add(Dense(2))
model.add(keras.layers.TimeDistributed(Dense(1)))
print(model.output_shape)

In [ ]:
model = Sequential()
model.add(LSTM(3, input_shape=(5, 1), return_sequences=False))
model.add(keras.layers.TimeDistributed(Dense(2)))
print(model.output_shape)

In [95]:
model = Sequential()
model.add(LSTM(32, input_shape=(10, 64),return_sequences=True))
print(model.output_shape)

(None, 10, 32)


In [59]:
keras.layers.TimeDistributed()

TypeError: __init__() missing 1 required positional argument: 'layer'